In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import gc
import xgboost as xgb
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression
import pickle
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor

In [61]:
transactions    = pd.read_csv('sales_train.csv.gz')
items           = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
test            = pd.read_csv('test.csv.gz')

In [62]:
INDEX_COLS = ['item_id', 'shop_id', 'date_block_num']

In [63]:
transactions = pd.merge(transactions, items, on='item_id', how='left')
transactions = transactions.drop('item_name', axis=1)
transactions = transactions[transactions['date_block_num'] > 12]
transactions.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
1366911,01.02.2014,13,27,15242,699.0,1.0,63
1366912,25.02.2014,13,27,15200,299.0,1.0,69
1366913,19.02.2014,13,27,15279,799.0,1.0,63
1366914,26.02.2014,13,27,15202,299.0,1.0,69
1366915,01.02.2014,13,27,14888,549.0,1.0,55


In [ ]:
len(transactions)

In [ ]:
np.append(test_item_ids,train_item_ids)

In [42]:
train_item_ids = transactions['item_id'].unique()
train_shop_ids = transactions['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = transactions['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))

In [108]:
items[['item_id', 'item_category_id']].to_dict('dict')

{'item_id': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51,
  52: 52,
  53: 53,
  54: 54,
  55: 55,
  56: 56,
  57: 57,
  58: 58,
  59: 59,
  60: 60,
  61: 61,
  62: 62,
  63: 63,
  64: 64,
  65: 65,
  66: 66,
  67: 67,
  68: 68,
  69: 69,
  70: 70,
  71: 71,
  72: 72,
  73: 73,
  74: 74,
  75: 75,
  76: 76,
  77: 77,
  78: 78,
  79: 79,
  80: 80,
  81: 81,
  82: 82,
  83: 83,
  84: 84,
  85: 85,
  86: 86,
  87: 87,
  88: 88,
  89: 89,
  90: 90,
  91: 91,
  92: 92,
  93: 93,
  94: 94,
  95: 95,
  96: 96,
  97: 97,
  98: 98,
  99: 99,
  100: 10

In [46]:
combinations = []
for shop in all_shop_ids:
    #get all article ids ever associated to this shop
    train_ids = transactions[transactions['shop_id'] == shop]['item_id'].unique()
    test_ids = test[test['shop_id'] == shop]['item_id'].unique()
    all_shop = np.unique(np.append(train_ids, test_ids))
    all_shop_combo = [[item, shop, block] for item in all_shop for block in train_blocks]
    for combo in all_shop_combo:
        combinations.append(combo)

In [45]:
len(combinations)

7722666

In [47]:
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['item_id', 'shop_id', 'date_block_num'])

In [48]:
len(all_combos)

7722666

In [110]:
all_combos.head()

,item_id,shop_id,date_block_num,item_category_id
0,0,54,13,40
1,0,54,14,40
2,0,54,15,40
3,0,54,16,40
4,0,54,17,40


In [109]:
all_combos = pd.merge(all_combos, items[['item_id', 'item_category_id']], on='item_id', how='left')

In [64]:
transactions['y'] = transactions.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].transform(np.sum).clip(0,40)

In [65]:
gc.collect()

transactions['item_cnt_sum'] = transactions.groupby(['item_id', 'date_block_num'])['item_cnt_day'].transform(np.sum)
transactions['item_cnt_mean'] = transactions.groupby(['item_id', 'date_block_num'])['item_cnt_day'].transform(np.mean)
transactions['item_price_mean'] = transactions.groupby(['item_id', 'date_block_num'])['item_price'].transform(np.mean)


transactions['shop_cnt_sum'] = transactions.groupby(['shop_id', 'date_block_num'])['item_cnt_day'].transform(np.sum)
transactions['shop_cnt_mean'] = transactions.groupby(['shop_id', 'date_block_num'])['item_cnt_day'].transform(np.mean)
transactions['shop_price_mean'] = transactions.groupby(['shop_id', 'date_block_num'])['item_price'].transform(np.mean)

transactions['category_cnt_sum'] = transactions.groupby(['item_category_id', 'date_block_num'])['item_cnt_day'].transform(np.sum)
transactions['category_cnt_mean'] = transactions.groupby(['item_category_id', 'date_block_num'])['item_cnt_day'].transform(np.mean)
transactions['category_price_mean'] = transactions.groupby(['item_category_id', 'date_block_num'])['item_price'].transform(np.mean)

In [52]:
len(transactions)

868080

In [66]:
transactions = transactions.drop_duplicates(INDEX_COLS)

In [54]:
len(transactions)

868080

In [101]:
transactions['item_category_id'].nunique()

78

In [111]:
training = pd.merge(all_combos,transactions,on=['item_id', 'shop_id', 'date_block_num'],how='left')

In [121]:
training.drop('item_category_id_y', inplace=True, axis=1)

KeyError: "labels ['item_category_id_y'] not contained in axis"

In [120]:
training.columns

Index(['item_id', 'shop_id', 'date_block_num', 'item_category_id_x', 'date',
       'item_price', 'item_cnt_day', 'y', 'item_cnt_sum', 'item_cnt_mean',
       'item_price_mean', 'shop_cnt_sum', 'shop_cnt_mean', 'shop_price_mean',
       'category_cnt_sum', 'category_cnt_mean', 'category_price_mean'],
      dtype='object')

In [126]:
training.rename(columns={'item_category_id_x': 'item_category_id'}, inplace=True)

In [127]:
training.head()

,item_id,shop_id,date_block_num,item_category_id,date,item_price,item_cnt_day,y,item_cnt_sum,item_cnt_mean,item_price_mean,shop_cnt_sum,shop_cnt_mean,shop_price_mean,category_cnt_sum,category_cnt_mean,category_price_mean
0,0,54,13,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,54,14,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,54,15,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,54,16,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,54,17,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
training[['item_id', 'shop_id', 'item_category_id', 'date_block_num']].sample(100)

,item_id,shop_id,item_category_id,date_block_num
6785834,19341,15,49,33
1345246,4117,39,58,20
5993531,17027,7,55,18
5491649,15770,57,38,15
2853537,8625,59,40,28
7074201,20252,31,40,28
3040754,9203,4,61,30
7402553,21265,28,61,24
5150924,15031,58,49,15
3217396,9870,5,40,20


In [129]:
training['y'] = training['y'].fillna(0)

In [130]:
transactions[(transactions['item_id'] == 15266) & (transactions['shop_id'] == 5)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,y,item_cnt_sum,item_cnt_mean,item_price_mean,shop_cnt_sum,shop_cnt_mean,shop_price_mean,category_cnt_sum,category_cnt_mean,category_price_mean
2332910,24.01.2015,24,5,15266,699.0,1.0,63,1.0,23.0,1.0,692.956522,810.0,1.033163,1120.071696,1476.0,1.035789,890.800954
2527492,21.03.2015,26,5,15266,699.0,1.0,63,1.0,21.0,1.0,682.857143,702.0,1.035398,833.320575,1359.0,1.026435,930.227372


In [131]:
training[(training['item_id'] == 15266) & (training['shop_id'] == 5)]

,item_id,shop_id,date_block_num,item_category_id,date,item_price,item_cnt_day,y,item_cnt_sum,item_cnt_mean,item_price_mean,shop_cnt_sum,shop_cnt_mean,shop_price_mean,category_cnt_sum,category_cnt_mean,category_price_mean
5266191,15266,5,13,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266192,15266,5,14,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266193,15266,5,15,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266194,15266,5,16,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266195,15266,5,17,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266196,15266,5,18,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266197,15266,5,19,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266198,15266,5,20,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266199,15266,5,21,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5266200,15266,5,22,63,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
len(training)

7722666

In [133]:
training = training.sample(frac=1)

In [134]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

training_int = training.select_dtypes(include=['int'])
converted_int = training_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(training_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([training_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

58.92 MB
58.92 MB


,before,after


In [135]:
training_float = training.select_dtypes(include=['float'])
converted_float = training_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(training_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([training_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

765.95 MB
412.43 MB


,before,after
float32,NaN,12.0
float64,12.0,NaN


In [136]:
optimized_training = training.copy()

optimized_training[converted_int.columns] = converted_int
optimized_training[converted_float.columns] = converted_float

print(mem_usage(training))
print(mem_usage(optimized_training))

1266.28 MB
912.76 MB


In [137]:
training = optimized_training
del optimized_training
gc.collect()

707

In [68]:
lag_columns = training.columns[7:]
lags = [1,2,3,6,12]

In [80]:
lag_columns = training.columns[8:17]

In [69]:
lag_columns

Index(['y', 'item_cnt_sum', 'item_cnt_mean', 'item_price_mean', 'shop_cnt_sum',
       'shop_cnt_mean', 'shop_price_mean', 'category_cnt_sum',
       'category_cnt_mean', 'category_price_mean', 'item_cnt_sum_lag_1',
       'item_cnt_mean_lag_1', 'item_price_mean_lag_1', 'shop_cnt_sum_lag_1',
       'shop_cnt_mean_lag_1', 'shop_price_mean_lag_1',
       'category_cnt_sum_lag_1', 'category_cnt_mean_lag_1',
       'category_price_mean_lag_1', 'item_cnt_sum_lag_2',
       'item_cnt_mean_lag_2', 'item_price_mean_lag_2', 'shop_cnt_sum_lag_2',
       'shop_cnt_mean_lag_2', 'shop_price_mean_lag_2',
       'category_cnt_sum_lag_2', 'category_cnt_mean_lag_2',
       'category_price_mean_lag_2', 'item_cnt_sum_lag_3',
       'item_cnt_mean_lag_3', 'item_price_mean_lag_3', 'shop_cnt_sum_lag_3',
       'shop_cnt_mean_lag_3', 'shop_price_mean_lag_3',
       'category_cnt_sum_lag_3', 'category_cnt_mean_lag_3',
       'category_price_mean_lag_3', 'item_cnt_sum_lag_6',
       'item_cnt_mean_lag_6', 'ite

In [139]:
training.head()

,item_id,shop_id,date_block_num,item_category_id,date,item_price,item_cnt_day,y,item_cnt_sum,item_cnt_mean,item_price_mean,shop_cnt_sum,shop_cnt_mean,shop_price_mean,category_cnt_sum,category_cnt_mean,category_price_mean
5091495,14845,15,16,65,03.05.2014,229.0,2.0,2.0,48.0,1.92,227.167999,938.0,1.111374,702.385681,880.0,1.222222,322.151093
5099338,14866,25,26,37,11.03.2015,399.0,1.0,1.0,6.0,1.00,399.000000,2678.0,1.106154,840.005981,3439.0,1.040545,392.435455
1046784,3347,54,31,23,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1782576,5380,7,25,75,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4980844,14494,24,14,41,01.03.2014,148.0,1.0,1.0,4.0,1.00,135.500000,1005.0,1.062368,871.382324,616.0,1.011494,1080.159424


In [140]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

merge_columns = ['lagged_block','item_id','shop_id']

for lag in lags:
    print(lag)
    lagged = transactions.copy()
    lagged.rename(columns={'date_block_num':'lagged_block'},inplace=True)
    training['lagged_block'] = training['date_block_num'] - lag
    lagged_names = [lagged_name(c,lag) for c in lag_columns]
    lag_mapping = dict(zip(lag_columns, lagged_names))
    lagged.rename(columns=lag_mapping,inplace=True)
    training = pd.merge(training,lagged[lagged_names+merge_columns],on=merge_columns,how='left')
    del lagged
    gc.collect()

1
2
3
6
12


In [141]:
len(training)

7722666

In [142]:
training.sample(10)

,item_id,shop_id,date_block_num,item_category_id,date,item_price,item_cnt_day,y,item_cnt_sum,item_cnt_mean,...,y_lag_12,item_cnt_sum_lag_12,item_cnt_mean_lag_12,item_price_mean_lag_12,shop_cnt_sum_lag_12,shop_cnt_mean_lag_12,shop_price_mean_lag_12,category_cnt_sum_lag_12,category_cnt_mean_lag_12,category_price_mean_lag_12
2688817,1384,22,29,30,08.06.2015,248.0,1.0,1.0,27.0,1.173913,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1646559,20072,12,17,37,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7127129,11172,28,24,40,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6895533,3787,50,14,55,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3024749,16533,42,30,37,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6464168,16174,34,24,65,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4346929,16560,44,14,40,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6199640,2642,53,24,55,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4815175,12705,12,18,40,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3463591,14112,26,15,37,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
training['date_block_num'].unique()

array([26, 16, 24, 15, 23, 33, 30, 21, 32, 25, 31, 20, 19, 18, 22, 17, 29,
       28, 27, 14, 13], dtype=int64)

In [ ]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training[(training['item_id'] == 30) & (training['shop_id'] == 30)].sort_values(by='date_block_num')

In [143]:
training.drop(columns=['y_lag_1', 'y_lag_2', 'y_lag_3', 'y_lag_6', 'y_lag_12', 'lagged_block'],inplace=True)

In [70]:
import pickle as pickle

#pickle.dump(training, open( "training", "wb"), protocol=4)

training = pickle.load( open( "training", "rb" ) )

In [7]:
val = training[training['date_block_num'].isin([32,33])]

In [26]:
val = training[training['date_block_num'].isin([32,33])]
non_zeros = val[val['y'] != 0]
zeros = val[val['y'] == 0]
zeros_keep = zeros.sample(int(len(non_zeros)*0.5))
val_indices = non_zeros.append(zeros_keep).index

In [27]:
len(val_indices)

91692

In [30]:
print(len(non_zeros))
print(len(zeros_keep))

61128
30564


In [14]:
lgbm_features = training.columns[17:]
lgbm_features = np.append( ['item_id', 'shop_id', 'item_category_id', 'date_block_num'], lgbm_features)
lgbm_features

array(['item_id', 'shop_id', 'item_category_id', 'date_block_num',
       'item_cnt_sum_lag_1', 'item_cnt_mean_lag_1',
       'item_price_mean_lag_1', 'shop_cnt_sum_lag_1',
       'shop_cnt_mean_lag_1', 'shop_price_mean_lag_1',
       'category_cnt_sum_lag_1', 'category_cnt_mean_lag_1',
       'category_price_mean_lag_1', 'item_cnt_sum_lag_2',
       'item_cnt_mean_lag_2', 'item_price_mean_lag_2',
       'shop_cnt_sum_lag_2', 'shop_cnt_mean_lag_2',
       'shop_price_mean_lag_2', 'category_cnt_sum_lag_2',
       'category_cnt_mean_lag_2', 'category_price_mean_lag_2',
       'item_cnt_sum_lag_3', 'item_cnt_mean_lag_3',
       'item_price_mean_lag_3', 'shop_cnt_sum_lag_3',
       'shop_cnt_mean_lag_3', 'shop_price_mean_lag_3',
       'category_cnt_sum_lag_3', 'category_cnt_mean_lag_3',
       'category_price_mean_lag_3', 'item_cnt_sum_lag_6',
       'item_cnt_mean_lag_6', 'item_price_mean_lag_6',
       'shop_cnt_sum_lag_6', 'shop_cnt_mean_lag_6',
       'shop_price_mean_lag_6', 'categor

In [10]:
categorical_features_indices = [0,1,2,3]

In [71]:
training[['item_id', 'shop_id', 'item_category_id', 'date_block_num']].sample(100)

,item_id,shop_id,item_category_id,date_block_num
2228818,14339,28,40,19
1102416,169,55,44,33
7010964,11576,51,40,23
5885848,7806,57,19,24
6818254,12040,48,40,13
2120499,2625,50,55,32
3403373,1856,39,19,25
4620353,17167,7,40,19
3042962,18907,42,40,13
2029416,13205,6,64,13


In [28]:
x = training
y = training['y']


x_train = x[~x.index.isin(val_indices)]
y_train = y[~y.index.isin(val_indices)]

x_val = x[x.index.isin(val_indices)]
y_val = y[y.index.isin(val_indices)]

In [52]:
x_train = training[training['date_block_num'] < 32]

non_zeros_train = x_train[x_train['y'] != 0]
zeros_train = x_train[x_train['y'] == 0]
zeros_keep_train = zeros_train.sample(int(len(non_zeros_train)*0.1))
x_train = non_zeros_train.append(zeros_keep_train)

y_train = x_train['y']


val = training[training['date_block_num'].isin([32,33])]
non_zeros_val = val[val['y'] != 0]
zeros_val = val[val['y'] == 0]
zeros_keep_val = zeros_val.sample(int(len(non_zeros_val)*0.5))
x_val = non_zeros_val.append(zeros_keep_val)
y_val = x_val['y']


In [35]:
len(x_val)

91692

In [77]:
x_train.columns

Index(['item_id', 'shop_id', 'date_block_num', 'item_category_id', 'date',
       'item_price', 'item_cnt_day', 'y', 'item_cnt_sum', 'item_cnt_mean',
       'item_price_mean', 'shop_cnt_sum', 'shop_cnt_mean', 'shop_price_mean',
       'category_cnt_sum', 'category_cnt_mean', 'category_price_mean',
       'item_cnt_sum_lag_1', 'item_cnt_mean_lag_1', 'item_price_mean_lag_1',
       'shop_cnt_sum_lag_1', 'shop_cnt_mean_lag_1', 'shop_price_mean_lag_1',
       'category_cnt_sum_lag_1', 'category_cnt_mean_lag_1',
       'category_price_mean_lag_1', 'item_cnt_sum_lag_2',
       'item_cnt_mean_lag_2', 'item_price_mean_lag_2', 'shop_cnt_sum_lag_2',
       'shop_cnt_mean_lag_2', 'shop_price_mean_lag_2',
       'category_cnt_sum_lag_2', 'category_cnt_mean_lag_2',
       'category_price_mean_lag_2', 'item_cnt_sum_lag_3',
       'item_cnt_mean_lag_3', 'item_price_mean_lag_3', 'shop_cnt_sum_lag_3',
       'shop_cnt_mean_lag_3', 'shop_price_mean_lag_3',
       'category_cnt_sum_lag_3', 'category_cnt_

In [36]:
gc.collect()
lgtrain = lgbm.Dataset(x_train[lgbm_features], label=y_train)
lgval = lgbm.Dataset(x_val[lgbm_features], label=y_val)



#[0.00542047893814942, 29, 24, 0.39949465609514856, 1, 0.67943500, 10]
params = {
        "num_threads": 16,
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        #"max_bin": 10,#default 255
        #"num_leaves": 10, #default 31
        #"bagging_fraction": 0.3,
        #"bagging_freq": 1,
        #"min_data_in_leaf": 50000,
        #"feature_fraction": 0.5,
        #"min_gain_to_split": 1,
       # "lambda_l1": 100,
        #"lambda_l2": 100,
        #"max_depth": 3, #default -1
        #"min_gain_to_split": 10,
        "learning_rate" : 0.02,
        #"histogram_pool_size": 1000,
        "categorical_column": categorical_features_indices
}


evals_result = {}
model_lgb = lgbm.train(params, lgtrain, 1000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=100, 
                      evals_result=evals_result)



Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.999412
Early stopping, best iteration is:
[38]	valid_0's rmse: 0.991213


In [ ]:
len()

In [54]:
cb_model = CatBoostRegressor(iterations=5000,
                             learning_rate=0.0001,
                             eval_metric='RMSE',
                             #thread_count=16,
                             task_type = "GPU",
                             use_best_model=True,
                             #l2_leaf_reg = 1000,
                            od_type = "Iter",
                            od_wait = 100,
                             #random_strength = 10,
                             #bagging_temperature = 1,
                             #one_hot_max_size = 2,
                             random_seed = 23)


cb_model.fit(x_train[lgbm_features], y_train, cat_features=categorical_features_indices,
             eval_set=(x_val[lgbm_features],y_val),
             #cat_features=categorical_features_pos,         
             verbose=True)

0:	learn: 1.2957909	test: 1.2610163	best: 1.2610163 (0)	total: 96ms	remaining: 8m
1:	learn: 1.2957133	test: 1.2609707	best: 1.2609707 (1)	total: 176ms	remaining: 7m 19s
2:	learn: 1.2956286	test: 1.2609195	best: 1.2609195 (2)	total: 270ms	remaining: 7m 29s
3:	learn: 1.2955468	test: 1.2608682	best: 1.2608682 (3)	total: 352ms	remaining: 7m 19s
4:	learn: 1.2954654	test: 1.2608153	best: 1.2608153 (4)	total: 432ms	remaining: 7m 11s
5:	learn: 1.2953818	test: 1.2607692	best: 1.2607692 (5)	total: 512ms	remaining: 7m 5s
6:	learn: 1.2953003	test: 1.2607238	best: 1.2607238 (6)	total: 591ms	remaining: 7m 1s
7:	learn: 1.2952175	test: 1.2606733	best: 1.2606733 (7)	total: 668ms	remaining: 6m 56s
8:	learn: 1.2951357	test: 1.2606203	best: 1.2606203 (8)	total: 747ms	remaining: 6m 54s
9:	learn: 1.2950524	test: 1.2605793	best: 1.2605793 (9)	total: 825ms	remaining: 6m 51s
10:	learn: 1.2949705	test: 1.2605267	best: 1.2605267 (10)	total: 905ms	remaining: 6m 50s
11:	learn: 1.2948885	test: 1.2604689	best: 1.260

93:	learn: 1.2881808	test: 1.2565318	best: 1.2565318 (93)	total: 7.53s	remaining: 6m 33s
94:	learn: 1.2880996	test: 1.2564794	best: 1.2564794 (94)	total: 7.61s	remaining: 6m 32s
95:	learn: 1.2880175	test: 1.2564274	best: 1.2564274 (95)	total: 7.68s	remaining: 6m 32s
96:	learn: 1.2879377	test: 1.2563751	best: 1.2563751 (96)	total: 7.77s	remaining: 6m 32s
97:	learn: 1.2878561	test: 1.2563297	best: 1.2563297 (97)	total: 7.85s	remaining: 6m 32s
98:	learn: 1.2877744	test: 1.2562747	best: 1.2562747 (98)	total: 7.94s	remaining: 6m 32s
99:	learn: 1.2876944	test: 1.2562354	best: 1.2562354 (99)	total: 8.02s	remaining: 6m 32s
100:	learn: 1.2876122	test: 1.2561913	best: 1.2561913 (100)	total: 8.1s	remaining: 6m 32s
101:	learn: 1.2875313	test: 1.2561562	best: 1.2561562 (101)	total: 8.18s	remaining: 6m 32s
102:	learn: 1.2874505	test: 1.2561111	best: 1.2561111 (102)	total: 8.26s	remaining: 6m 32s
103:	learn: 1.2873682	test: 1.2560664	best: 1.2560664 (103)	total: 8.35s	remaining: 6m 32s
104:	learn: 1.

186:	learn: 1.2806669	test: 1.2522642	best: 1.2522642 (186)	total: 14.9s	remaining: 6m 24s
187:	learn: 1.2805860	test: 1.2522136	best: 1.2522136 (187)	total: 15s	remaining: 6m 24s
188:	learn: 1.2805068	test: 1.2521777	best: 1.2521777 (188)	total: 15.1s	remaining: 6m 23s
189:	learn: 1.2804259	test: 1.2521330	best: 1.2521330 (189)	total: 15.2s	remaining: 6m 23s
190:	learn: 1.2803458	test: 1.2520973	best: 1.2520973 (190)	total: 15.2s	remaining: 6m 23s
191:	learn: 1.2802652	test: 1.2520530	best: 1.2520530 (191)	total: 15.3s	remaining: 6m 23s
192:	learn: 1.2801856	test: 1.2520031	best: 1.2520031 (192)	total: 15.4s	remaining: 6m 23s
193:	learn: 1.2801048	test: 1.2519592	best: 1.2519592 (193)	total: 15.5s	remaining: 6m 23s
194:	learn: 1.2800247	test: 1.2519100	best: 1.2519100 (194)	total: 15.6s	remaining: 6m 23s
195:	learn: 1.2799450	test: 1.2518612	best: 1.2518612 (195)	total: 15.6s	remaining: 6m 23s
196:	learn: 1.2798646	test: 1.2518109	best: 1.2518109 (196)	total: 15.7s	remaining: 6m 23s
1

279:	learn: 1.2732503	test: 1.2480430	best: 1.2480430 (279)	total: 22.5s	remaining: 6m 19s
280:	learn: 1.2731710	test: 1.2480081	best: 1.2480081 (280)	total: 22.6s	remaining: 6m 19s
281:	learn: 1.2730919	test: 1.2479732	best: 1.2479732 (281)	total: 22.7s	remaining: 6m 19s
282:	learn: 1.2730127	test: 1.2479295	best: 1.2479295 (282)	total: 22.7s	remaining: 6m 19s
283:	learn: 1.2729330	test: 1.2478796	best: 1.2478796 (283)	total: 22.8s	remaining: 6m 18s
284:	learn: 1.2728546	test: 1.2478308	best: 1.2478308 (284)	total: 22.9s	remaining: 6m 19s
285:	learn: 1.2727749	test: 1.2477782	best: 1.2477782 (285)	total: 23s	remaining: 6m 18s
286:	learn: 1.2726957	test: 1.2477340	best: 1.2477340 (286)	total: 23.1s	remaining: 6m 18s
287:	learn: 1.2726165	test: 1.2476995	best: 1.2476995 (287)	total: 23.1s	remaining: 6m 18s
288:	learn: 1.2725369	test: 1.2476563	best: 1.2476563 (288)	total: 23.2s	remaining: 6m 18s
289:	learn: 1.2724582	test: 1.2476077	best: 1.2476077 (289)	total: 23.3s	remaining: 6m 18s
2

372:	learn: 1.2659196	test: 1.2439277	best: 1.2439277 (372)	total: 29.9s	remaining: 6m 11s
373:	learn: 1.2658405	test: 1.2438851	best: 1.2438851 (373)	total: 30s	remaining: 6m 11s
374:	learn: 1.2657629	test: 1.2438421	best: 1.2438421 (374)	total: 30.1s	remaining: 6m 11s
375:	learn: 1.2656842	test: 1.2438037	best: 1.2438037 (375)	total: 30.2s	remaining: 6m 11s
376:	learn: 1.2656055	test: 1.2437650	best: 1.2437650 (376)	total: 30.3s	remaining: 6m 10s
377:	learn: 1.2655278	test: 1.2437222	best: 1.2437222 (377)	total: 30.3s	remaining: 6m 10s
378:	learn: 1.2654488	test: 1.2436831	best: 1.2436831 (378)	total: 30.4s	remaining: 6m 10s
379:	learn: 1.2653708	test: 1.2436406	best: 1.2436406 (379)	total: 30.5s	remaining: 6m 10s
380:	learn: 1.2652922	test: 1.2435975	best: 1.2435975 (380)	total: 30.6s	remaining: 6m 10s
381:	learn: 1.2652143	test: 1.2435603	best: 1.2435603 (381)	total: 30.6s	remaining: 6m 10s
382:	learn: 1.2651349	test: 1.2435173	best: 1.2435173 (382)	total: 30.7s	remaining: 6m 10s
3

...

4598:	learn: 1.0167407	test: 1.1238468	best: 1.1238468 (4598)	total: 6m 3s	remaining: 31.7s
4599:	learn: 1.0166974	test: 1.1238313	best: 1.1238313 (4599)	total: 6m 3s	remaining: 31.6s
4600:	learn: 1.0166548	test: 1.1238141	best: 1.1238141 (4600)	total: 6m 3s	remaining: 31.5s
4601:	learn: 1.0166128	test: 1.1237906	best: 1.1237906 (4601)	total: 6m 3s	remaining: 31.4s
4602:	learn: 1.0165701	test: 1.1237732	best: 1.1237732 (4602)	total: 6m 3s	remaining: 31.3s
4603:	learn: 1.0165274	test: 1.1237536	best: 1.1237536 (4603)	total: 6m 3s	remaining: 31.3s
4604:	learn: 1.0164849	test: 1.1237390	best: 1.1237390 (4604)	total: 6m 3s	remaining: 31.2s
4605:	learn: 1.0164429	test: 1.1237215	best: 1.1237215 (4605)	total: 6m 3s	remaining: 31.1s
4606:	learn: 1.0164006	test: 1.1237033	best: 1.1237033 (4606)	total: 6m 3s	remaining: 31s
4607:	learn: 1.0163582	test: 1.1236865	best: 1.1236865 (4607)	total: 6m 3s	remaining: 30.9s
4608:	learn: 1.0163150	test: 1.1236719	best: 1.1236719 (4608)	total: 6m 3s	remaini

4689:	learn: 1.0128943	test: 1.1222347	best: 1.1222347 (4689)	total: 6m 10s	remaining: 24.5s
4690:	learn: 1.0128523	test: 1.1222168	best: 1.1222168 (4690)	total: 6m 10s	remaining: 24.4s
4691:	learn: 1.0128103	test: 1.1222013	best: 1.1222013 (4691)	total: 6m 10s	remaining: 24.3s
4692:	learn: 1.0127682	test: 1.1221863	best: 1.1221863 (4692)	total: 6m 10s	remaining: 24.2s
4693:	learn: 1.0127265	test: 1.1221633	best: 1.1221633 (4693)	total: 6m 10s	remaining: 24.1s
4694:	learn: 1.0126848	test: 1.1221523	best: 1.1221523 (4694)	total: 6m 10s	remaining: 24.1s
4695:	learn: 1.0126425	test: 1.1221379	best: 1.1221379 (4695)	total: 6m 10s	remaining: 24s
4696:	learn: 1.0126009	test: 1.1221155	best: 1.1221155 (4696)	total: 6m 10s	remaining: 23.9s
4697:	learn: 1.0125588	test: 1.1220984	best: 1.1220984 (4697)	total: 6m 10s	remaining: 23.8s
4698:	learn: 1.0125169	test: 1.1220808	best: 1.1220808 (4698)	total: 6m 10s	remaining: 23.7s
4699:	learn: 1.0124747	test: 1.1220637	best: 1.1220637 (4699)	total: 6m 

4778:	learn: 1.0091864	test: 1.1206511	best: 1.1206511 (4778)	total: 6m 17s	remaining: 17.4s
4779:	learn: 1.0091453	test: 1.1206368	best: 1.1206368 (4779)	total: 6m 17s	remaining: 17.4s
4780:	learn: 1.0091041	test: 1.1206209	best: 1.1206209 (4780)	total: 6m 17s	remaining: 17.3s
4781:	learn: 1.0090633	test: 1.1206032	best: 1.1206032 (4781)	total: 6m 17s	remaining: 17.2s
4782:	learn: 1.0090214	test: 1.1205860	best: 1.1205860 (4782)	total: 6m 17s	remaining: 17.1s
4783:	learn: 1.0089804	test: 1.1205648	best: 1.1205648 (4783)	total: 6m 17s	remaining: 17.1s
4784:	learn: 1.0089399	test: 1.1205552	best: 1.1205552 (4784)	total: 6m 17s	remaining: 17s
4785:	learn: 1.0088990	test: 1.1205376	best: 1.1205376 (4785)	total: 6m 17s	remaining: 16.9s
4786:	learn: 1.0088573	test: 1.1205232	best: 1.1205232 (4786)	total: 6m 17s	remaining: 16.8s
4787:	learn: 1.0088167	test: 1.1205110	best: 1.1205110 (4787)	total: 6m 17s	remaining: 16.7s
4788:	learn: 1.0087751	test: 1.1204938	best: 1.1204938 (4788)	total: 6m 

4868:	learn: 1.0055011	test: 1.1190850	best: 1.1190850 (4868)	total: 6m 24s	remaining: 10.3s
4869:	learn: 1.0054604	test: 1.1190712	best: 1.1190712 (4869)	total: 6m 24s	remaining: 10.3s
4870:	learn: 1.0054188	test: 1.1190493	best: 1.1190493 (4870)	total: 6m 24s	remaining: 10.2s
4871:	learn: 1.0053779	test: 1.1190354	best: 1.1190354 (4871)	total: 6m 24s	remaining: 10.1s
4872:	learn: 1.0053378	test: 1.1190179	best: 1.1190179 (4872)	total: 6m 24s	remaining: 10s
4873:	learn: 1.0052973	test: 1.1189974	best: 1.1189974 (4873)	total: 6m 24s	remaining: 9.94s
4874:	learn: 1.0052564	test: 1.1189794	best: 1.1189794 (4874)	total: 6m 24s	remaining: 9.87s
4875:	learn: 1.0052159	test: 1.1189657	best: 1.1189657 (4875)	total: 6m 24s	remaining: 9.79s
4876:	learn: 1.0051751	test: 1.1189513	best: 1.1189513 (4876)	total: 6m 24s	remaining: 9.71s
4877:	learn: 1.0051336	test: 1.1189309	best: 1.1189309 (4877)	total: 6m 24s	remaining: 9.63s
4878:	learn: 1.0050923	test: 1.1189166	best: 1.1189166 (4878)	total: 6m 

4958:	learn: 1.0018605	test: 1.1175910	best: 1.1175910 (4958)	total: 6m 31s	remaining: 3.24s
4959:	learn: 1.0018199	test: 1.1175769	best: 1.1175769 (4959)	total: 6m 31s	remaining: 3.16s
4960:	learn: 1.0017804	test: 1.1175653	best: 1.1175653 (4960)	total: 6m 31s	remaining: 3.08s
4961:	learn: 1.0017414	test: 1.1175380	best: 1.1175380 (4961)	total: 6m 31s	remaining: 3s
4962:	learn: 1.0017007	test: 1.1175233	best: 1.1175233 (4962)	total: 6m 31s	remaining: 2.92s
4963:	learn: 1.0016604	test: 1.1175094	best: 1.1175094 (4963)	total: 6m 31s	remaining: 2.84s
4964:	learn: 1.0016206	test: 1.1174917	best: 1.1174917 (4964)	total: 6m 31s	remaining: 2.76s
4965:	learn: 1.0015804	test: 1.1174754	best: 1.1174754 (4965)	total: 6m 31s	remaining: 2.68s
4966:	learn: 1.0015398	test: 1.1174569	best: 1.1174569 (4966)	total: 6m 32s	remaining: 2.6s
4967:	learn: 1.0014991	test: 1.1174467	best: 1.1174467 (4967)	total: 6m 32s	remaining: 2.52s
4968:	learn: 1.0014601	test: 1.1174296	best: 1.1174296 (4968)	total: 6m 32

In [47]:
len(training.item_category_id.unique())

79

In [ ]:
print('Plotting feature importances...')
ax = lgbm.plot_importance(model_lgb, max_num_features=10)
plt.show()

In [ ]:
best_features_indices = np.argsort(model_lgb.feature_importance())[::-1][0:5]
lgbm_features[best_features_indices]

In [72]:
test_w_cat_ids = test.set_index('item_id').join(items.set_index('item_id'))
test_w_cat_ids['date_block_num'] = 34

In [81]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

merge_columns = ['lagged_block','item_id','shop_id']

for lag in lags:
    print(lag)
    lagged = transactions.copy()
    lagged.rename(columns={'date_block_num':'lagged_block'},inplace=True)
    test_w_cat_ids['lagged_block'] = test_w_cat_ids['date_block_num'] - lag
    lagged_names = [lagged_name(c,lag) for c in lag_columns]
    print(lag_columns)
    lag_mapping = dict(zip(lag_columns, lagged_names))
    lagged.rename(columns=lag_mapping,inplace=True)
    test_w_cat_ids = pd.merge(test_w_cat_ids,lagged[lagged_names+merge_columns],on=merge_columns,how='left')
    del lagged
    gc.collect()

1
Index(['item_cnt_sum', 'item_cnt_mean', 'item_price_mean', 'shop_cnt_sum',
       'shop_cnt_mean', 'shop_price_mean', 'category_cnt_sum',
       'category_cnt_mean', 'category_price_mean'],
      dtype='object')
2
Index(['item_cnt_sum', 'item_cnt_mean', 'item_price_mean', 'shop_cnt_sum',
       'shop_cnt_mean', 'shop_price_mean', 'category_cnt_sum',
       'category_cnt_mean', 'category_price_mean'],
      dtype='object')
3
Index(['item_cnt_sum', 'item_cnt_mean', 'item_price_mean', 'shop_cnt_sum',
       'shop_cnt_mean', 'shop_price_mean', 'category_cnt_sum',
       'category_cnt_mean', 'category_price_mean'],
      dtype='object')
6
Index(['item_cnt_sum', 'item_cnt_mean', 'item_price_mean', 'shop_cnt_sum',
       'shop_cnt_mean', 'shop_price_mean', 'category_cnt_sum',
       'category_cnt_mean', 'category_price_mean'],
      dtype='object')
12
Index(['item_cnt_sum', 'item_cnt_mean', 'item_price_mean', 'shop_cnt_sum',
       'shop_cnt_mean', 'shop_price_mean', 'category_cnt_sum',
   

In [29]:
test_w_cat_ids.head()

,ID,item_id,shop_id,item_name,item_category_id,date_block_num,lagged_block,y_lag_1,item_cnt_sum_lag_1,item_cnt_mean_lag_1,...,y_lag_12,item_cnt_sum_lag_12,item_cnt_mean_lag_12,item_price_mean_lag_12,shop_cnt_sum_lag_12,shop_cnt_mean_lag_12,shop_price_mean_lag_12,category_cnt_sum_lag_12,category_cnt_mean_lag_12,category_price_mean_lag_12
0,2587,30,5,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,34,22,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0000,0.000000
1,7687,30,4,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,34,22,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0000,0.000000
2,12787,30,6,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,34,22,0.0,0.0,0.0,...,1.0,13.0,1.0,169.0,3150.0,1.379159,1262.87496,14815.0,1.0903,264.032958
3,17887,30,3,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,34,22,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0000,0.000000
4,22987,30,2,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,34,22,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0000,0.000000


In [ ]:
len(test_w_cat_ids)

In [ ]:
preds = model_lgb.predict(test_w_cat_ids[lgbm_features])
preds.clip(0,20,out=preds)

In [82]:
preds = cb_model.predict(test_w_cat_ids[lgbm_features])
preds.clip(0,20,out=preds)

array([0.24522966, 0.24498314, 0.26768755, ..., 0.2434715 , 0.24369129,
       0.24326422])

In [83]:
np.sum(preds)

50095.25349830084

In [84]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds.astype(int)
#submission['item_cnt_month'] = ensemble_preds.astype(int)


submission.to_csv('submission.csv', index=False)